In [1]:
import re
import logging

import numpy as np
import pandas as pd
import spacy
import torch
import csv
from torchtext import data
import urllib.request

In [2]:
#The User data 
User_data = pd.read_csv('data/user-info.csv', usecols = ['user.id','doc.id','rating'])
User_data = User_data.rename(columns={'user.id': 'user_id','doc.id': 'doc_id'})


In [3]:
# The article dataset
article_data = pd.read_csv('data/raw-data.csv', usecols = ['doc.id','title','citeulike.id', 'raw.abstract'],encoding = "ISO-8859-1")
article_data = article_data.rename(columns={'raw.abstract': 'abstract','doc.id': 'doc_id','citeulike.id': 'citeulike_id'})


In [4]:
CiteULike_data=pd.merge(User_data,article_data,on="doc_id")

In [5]:
le = max(len(x) for x in CiteULike_data.abstract)
print(le)

122938


In [6]:
NLP = spacy.load('en')
MAX_CHARS = 1229381
VAL_RATIO = 0.2
LOGGER = logging.getLogger("CiteULike_data")

In [7]:
def tokenizer(abs_text):
    abs_text = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", 
        str(abs_text))
    abs_text = re.sub(r"[ ]+", " ", str(abs_text))
    abs_text = re.sub(r"\!+", "!", str(abs_text))
    abs_text = re.sub(r"\,+", ",", str(abs_text))
    abs_text = re.sub(r"\?+", "?", str(abs_text))
    if (len(abs_text) > MAX_CHARS):
        abs_text = abs_text[:MAX_CHARS]
    return [
        x.text for x in NLP.tokenizer(abs_text) if x.text != " "]

In [8]:
VAL_RATIO = 0.2

def prepare_csv(seed=999):
    df_train = CiteULike_data
    df_train["abstract"] = \
        df_train.abstract.str.replace("\n", " ")
    idx = np.arange(df_train.shape[0])
    np.random.seed(seed)
    np.random.shuffle(idx)
    val_size = int(len(idx) * VAL_RATIO)
    df_train.iloc[idx[val_size:], :].to_csv(
        "cache/dataset_train.csv", index=False)
    df_train.iloc[idx[:val_size], :].to_csv(
        "cache/dataset_val.csv", index=False)

In [9]:
def get_dataset(fix_length=100, lower=False, vectors=None):
    if vectors is not None:
        # pretrain vectors only supports all lower cases
        lower = True
    LOGGER.debug("Preparing CSV files...")
    prepare_csv()
    abs_text = data.Field(
        sequential=True,
        fix_length=fix_length,
        tokenize=tokenizer,
        pad_first=True,
        tensor_type=torch.LongTensor,
        lower=lower
    )
    LOGGER.debug("Reading train csv file...")
    train, val = data.TabularDataset.splits(
        path='cache/', format='csv', skip_header=True,
        train='dataset_train.csv', validation='dataset_val.csv',
        fields=[
            ('abstract', abs_text),
            ('doc_id', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.ByteTensor)),
            ('rating', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.ByteTensor)),
            ('title', None),
            ('citeulike_id', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.ByteTensor)),
            ('user_id', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.ByteTensor)),
        ])
    LOGGER.debug("Building vocabulary...")
    abs_text.build_vocab(
        train, val,
        max_size=20000,
        min_freq=50,
        vectors=vectors
    )
    LOGGER.debug("Done preparing the datasets")
    return train, val


In [14]:
def get_iterator(dataset, batch_size, train=True, shuffle=True, repeat=False):
    dataset_iter = data.Iterator(
        dataset, batch_size=batch_size, device=0,
        train=train, shuffle=shuffle, repeat=repeat,
        sort=True
    )
    return dataset_iter

In [17]:
x=get_iterator(
            CiteULike_data, 100, train=True,
            shuffle=True, repeat=False
        )

AttributeError: 'DataFrame' object has no attribute 'sort_key'

In [15]:
y = get_iterator(CiteULike_data, batch_size = 100)

AttributeError: 'DataFrame' object has no attribute 'sort_key'

In [16]:
len(CiteULike_data.doc_id.unique())

16980

In [17]:
len(tokenizer(CiteULike_data.abstract))

641

In [19]:
len(CiteULike_data.abstract)

204986

In [ ]:
prepare_csv(999)